## **REGRESSÃO POR VETORES DE SUPORTE (SVR)**

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html 

**Atributos previsores**

Cost ctr = centro de custo (setor da linha de produção)

Posting date = data de produção

QTY = quantidade de peças produzidas

**Variável alvo**

Employees QTY = quantidade de funcionários

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\TG1\zppoutput_processed.csv", delimiter=';')

In [3]:
df.head()

,Cost_ctr,Posting_date,QTY,Employees_QTY
0,0,0,1160,11
1,0,1,3330,13
2,0,2,1650,12
3,0,3,1320,18
4,0,4,1490,6


In [4]:
x1 = df.iloc[:,0:3].values
x1

array([[    0,     0,  1160],
       [    0,     1,  3330],
       [    0,     2,  1650],
       ...,
       [    5,  3610, 66944],
       [    5,  3611, 58424],
       [    5,  3612,   300]], dtype=int64)

In [5]:
y = df.iloc[:, 3].values
y

array([ 11,  13,  12, ..., 808, 528,   1], dtype=int64)

## **BASE DE TREINO E TESTE**

In [6]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x1, y, test_size = 0.3, random_state = 10)

In [7]:
x_treino.shape, y_treino.shape

((2529, 3), (2529,))

In [8]:
x_teste.shape, y_teste.shape

((1084, 3), (1084,))

In [9]:
# kernel = rbf, linear, polinomial (poly)
SVR = SVR(kernel='rbf')
SVR.fit(x_treino,y_treino)

SVR()

In [10]:
SVR.score(x_treino, y_treino)

0.8427594859390511

In [11]:
SVR.score(x_teste, y_teste)

0.8419645262870654

OBS: Este algorítmo depende muito de termos dados padronizados para funcionar bem, portanto faremos uma padronização de escala dos valores!

**Padronização de escala**

In [12]:
x_scaler = StandardScaler()
x_treino_scaler = x_scaler.fit_transform(x_treino)

In [13]:
y_scaler = StandardScaler()
y_treino_scaler = y_scaler.fit_transform(y_treino.reshape(-1,1))

In [14]:
x_teste_scaler = x_scaler.transform(x_teste)
x_teste_scaler

array([[-0.97507172, -1.16511355, -0.92126138],
       [-1.5765245 , -1.56781433, -0.99104657],
       [ 0.82928661,  0.93605963, -0.4456212 ],
       ...,
       [ 0.82928661,  0.67690531, -0.51305841],
       [-0.37361894, -0.23447013, -0.24207784],
       [-0.97507172, -0.76722977, -0.82811181]])

In [15]:
y_teste_scaler = y_scaler.transform(y_teste.reshape(-1,1))
y_teste_scaler

array([[-0.75521085],
       [-0.88790681],
       [-0.3305838 ],
       ...,
       [-0.38366218],
       [-0.40135498],
       [-0.78617324]])

In [17]:
SVR2 = SVR(kernel='rbf')
SVR2.fit(x_treino_scaler, y_treino_scaler.ravel())

# .ravel() é para retornar matriz 1D


SVR()

In [18]:
SVR2.score(x_treino_scaler, y_treino_scaler)

0.9420568359252781

**TESTE**

In [19]:
SVR2.score(x_teste_scaler, y_teste_scaler)

0.9469166675116377

In [20]:
previsoes_teste = SVR2.predict(x_teste_scaler)
previsoes_teste

array([-0.77072361, -0.80255973, -0.46218099, ..., -0.50003477,
       -0.36222843, -0.72840941])

**Revertendo a transformação**

In [21]:
y_teste_inverse = y_scaler.inverse_transform(y_teste_scaler)
previsoes_inverse = y_scaler.inverse_transform(previsoes_teste.reshape(-1, 1))

**Métricas de Desempenho**

In [22]:
# Erro quadrático médio
mean_squared_error(y_teste_inverse, previsoes_inverse)

2742.874584410188

In [23]:
# Raiz do erro quadrático médio (RMSE)
np.sqrt(mean_squared_error(y_teste_inverse, previsoes_inverse))

52.37246017145068

### **Validação Cruzada**

In [24]:
x = StandardScaler()
independente_scaler = x.fit_transform(x1)
independente_scaler

array([[-1.55862659, -1.73157148, -1.01383386],
       [-1.55862659, -1.73061269, -0.93019395],
       [-1.55862659, -1.7296539 , -0.99494743],
       ...,
       [ 1.43420816,  1.7296539 ,  1.52172748],
       [ 1.43420816,  1.73061269,  1.19333482],
       [ 1.43420816,  1.73157148, -1.04698148]])

In [25]:
z = StandardScaler()
dependente_scaler = z.fit_transform(y.reshape(-1,1))

In [26]:
# Separando os dados em folds
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [28]:
# Criando o modelo
modelo = SVR(kernel='rbf')
resultado = cross_val_score(modelo, independente_scaler, dependente_scaler.ravel(), cv = kfold)
resultado

array([0.95680532, 0.92096539, 0.95477439, 0.94452989, 0.92933112,
       0.95595564, 0.94988156, 0.9468501 , 0.9594153 , 0.92625832,
       0.93622244, 0.93653938, 0.93890066, 0.93706578, 0.94863463])

In [29]:
# Usamos a média e o desvio padrão
print("Coeficiente de Determinação médio: %.2f%%" % (resultado.mean() * 100.0))

Coeficiente de Determinação médio: 94.28%


**REGRESSÃO COM VETORES DE SUPORTE:** R^2 = 0,942/0,947; RMSE = 52,37; R^2 Validação Cruzada: 94,28%